In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, Image, HTML, Javascript
import cv2
import numpy as np
import matplotlib.pyplot as plt
import io
import base64
from base64 import b64decode
from base64 import b64encode  
import tempfile
import time
import os


# Load model and labels
config_file = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
frozen_model = 'frozen_inference_graph.pb'
model = cv2.dnn_DetectionModel(frozen_model, config_file)
model.setInputSize(320, 320)
model.setInputScale(1.0 / 127.5) # 255/2=127.5
model.setInputMean((127.5, 127.5, 127.5)) # mobilenet => [-1,1]
model.setInputSwapRB(True) # auto convert BGR to RGB

classLabels = []
file_name = 'Labels.txt'
with open(file_name, 'rt') as fpt:
    classLabels = fpt.read().rstrip('\n').split('\n')
    # print(classLabels)

# Function to handle image upload--------------
def handle_image_upload(change):
    if len(change['new']) > 0:
        uploaded_file = change['new']
        
        # Simulate progress with sleep
        for i in range(101):
            loader.value = i
            time.sleep(0.01)  # Sleep for 10 milliseconds for smooth progression

             # Update description when progress reaches maximum
            if i == 100:
                loader.description = 'Image Uploaded 100% !!!'  # Change description to 'Uploaded'
                
        # Extract image content
        image_content = uploaded_file[list(uploaded_file.keys())[0]]['content']
        
        # Display the uploaded image
        img_widget = widgets.Image(value=image_content, height='400px', width='600px')
        
        # Display the image
        with uploaded_output:
            clear_output()
            display(img_widget)
             
        # Reset the FileUpload widget to display "Upload(0)"
        image_button.value.clear()


# Function to handle detect button click
def handle_detect_button_click(b):
    # Display the uploaded image
    with uploaded_output:
        clear_output()
        uploaded_files = image_upload_button.value
        if uploaded_files:
            uploaded_file = uploaded_files[0]  # Access the first uploaded file
            image_content = uploaded_file['content']
            
            image = np.frombuffer(image_content, np.uint8)
            image = cv2.imdecode(image, cv2.IMREAD_COLOR)

            # Perform object detection on the image
            ClassIndex, confidence, bbox = model.detect(image, confThreshold=0.55)
    
            for ClassInd, conf, boxes in zip(ClassIndex.flatten(), confidence.flatten(), bbox):
                if ClassInd <= 80:
                    cv2.rectangle(image, (boxes[0], boxes[1]), (boxes[0]+boxes[2], boxes[1]+boxes[3]), (255, 0, 0), 2)
                    cv2.putText(image, classLabels[ClassInd-1], (boxes[0], boxes[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    
            # Convert image to base64 encoding
            _, buffer = cv2.imencode('.jpg', image)
            img_str = base64.b64encode(buffer).decode('utf-8')
    
            # Display annotated image
            display(Image(data=buffer, format='jpg'))
            


# Create buttons for image
image_upload_button = widgets.FileUpload(accept='.jpeg, .jpg, .png', description='Upload Image', multiple=False)

#Create label
# image_label=widgets.HTML(
#     value="<h2>Object Detection in IMAGE (Upload or Capture)</h2>"
# )

# Create detect button
detect_button = button = widgets.Button(
    description='Detect in Uploaded Image',
    disabled=False,
    button_style='Success',
    tooltip='Detect Image',
    icon='image', # (FontAwesome names without the `fa-` prefix)
    layout=widgets.Layout(width='200px')  # Set width to 200px
)

detect_button.on_click(handle_detect_button_click)


#Function to perform object detection in captured images
def handle_detect_captured_button_click(b):
    with captured_output:
        clear_output()
        
        # Capture image from webcam
        cap = cv2.VideoCapture(0)  # 0 for default webcam
        
        while True:
            # Capture frame-by-frame
            ret, frame = cap.read()
            
            # Display the resulting frame
            cv2.imshow('Capture Image in Webcam(Press Space to Capture Image)', frame)
            
            # Check for 'q' or 'Space' key press
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord(' '):
                # Capture image on spacebar press
                cv2.imwrite("captured_image.jpg", frame)
                print("🎉 Image captured successfully!!! 🎉")
                cap.release()
                cv2.destroyAllWindows()
                break
                
        # Perform object detection on the captured image
        image = cv2.imread("captured_image.jpg")
        ClassIndex, confidence, bbox = model.detect(image, confThreshold=0.55)
        
        for ClassInd, conf, boxes in zip(ClassIndex.flatten(), confidence.flatten(), bbox):
            if ClassInd <= 80:
                cv2.rectangle(image, (boxes[0], boxes[1]), (boxes[0]+boxes[2], boxes[1]+boxes[3]), (255, 0, 0), 2)
                if classLabels:  # Check if class labels are loaded
                    cv2.putText(image, classLabels[ClassInd-1], (boxes[0], boxes[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Convert image to base64 for display in captured_output widget
        _, buffer = cv2.imencode('.jpg', image)
        img_str = b64encode(buffer).decode('utf-8')
        
        # Display annotated image in captured_output widget
        # captured_img_widget = widgets.Image(value=image_content, height='400px', width='600px')
        # display(img_widget)

        # Display annotated image
        display(Image(data=buffer, format='jpg'))

# Create detect button
detect_capture_button = button = widgets.Button(
    description='Detect in Captured Image',
    disabled=False,
    button_style='Info',
    tooltip='Detect Image',
    icon='camera', 
    layout=widgets.Layout(width='200px')  # Set width to 200px
)

# Attach event handlers to the image upload button
image_upload_button.observe(handle_image_upload, names='value')

detect_capture_button.on_click(handle_detect_captured_button_click)



# Create loader
loader = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Uploading Image...',
    bar_style='success',
    style={'bar_color': 'red', 'description_width': 'initial'},
    orientation='horizontal',
    format='<font color="black">%(value)d%%</font>'  # Show progress percentage in black font color
)

# Create output widget for UPLOADED image
uploaded_output = widgets.Output(layout={'height': '400px', 'width': '600px'})

# Create output widget for CAPTURED Image
captured_output = widgets.Output(layout={'height': '400px', 'width': '600px'})


# Create container widget for image-related components
uploaded_image_tab_content = widgets.VBox([image_upload_button,loader, widgets.HBox([detect_button], layout=widgets.Layout(margin='10px 10px 0px 0px')), uploaded_output])

# Create container widget for image-related components
captured_image_tab_content = widgets.VBox([widgets.HBox([detect_capture_button], layout=widgets.Layout(margin='10px 10px 0px 0px')), captured_output])

image_tab_content = widgets.Accordion(children=[uploaded_image_tab_content, captured_image_tab_content], titles=('1. Object Detection in UPLOADED Image', '2. Object Detection in CAPTURED Image'))


# VIDEO DETECTION--------------------------------
# Define global variables
uploaded_video_content = None

# Function to handle VIDEO upload
def handle_video_upload(change):
    global uploaded_video_content

    if len(change['new']) > 0:
        uploaded_file = change['new'][0]  # Access the first element of the tuple
        uploaded_video_content = uploaded_file['content']
            
        # Simulate progress with sleep
        for i in range(101):
            video_loader.value = i
            time.sleep(0.02)  # Sleep for 20 milliseconds for smooth progression

            # Update description when progress reaches maximum
            if i == 100:
                video_loader.description = 'Video Uploaded 100% !!!'  # Change description to 'Uploaded'
                

# Function to handle detect video button click
def handle_detect_video_button_click(b):
    global uploaded_video_content

    if uploaded_video_content:
        # Save the video content to a temporary file
        with tempfile.NamedTemporaryFile(suffix='.mp4', delete=False) as tmp_file:
            tmp_file.write(uploaded_video_content)
            video_path = tmp_file.name

        # Create VideoCapture object for uploaded video
        capt = cv2.VideoCapture(video_path)

        #check if video opened correctly or not
        if not capt.isOpened():
            raise IOError("Cannot open Video") #replace with webcam for webcam detection
        
        font_scale= 3
        font= cv2.FONT_HERSHEY_PLAIN
        
        while True:
            ret, frame= capt.read()
           
            ClassIndex, confidence, bbox = model.detect(frame,confThreshold=0.55) #0.5= 50%
        
            print(ClassIndex)
            if(len(ClassIndex)!=0):
                    for ClassInd, conf, boxes in zip(ClassIndex.flatten(), confidence.flatten(),bbox): #all 3 variable in zip
                        if(ClassInd <=80):
                            cv2.rectangle(frame, boxes,(255,0,0),2) #red color rectangle adn text is green
                            cv2.putText(frame, classLabels[ClassInd-1],(boxes[0]+10,boxes[1]+40),font, fontScale=font_scale, color=(0,255,0), thickness=3)
            
            # Resize the frame to the desired width and height
            frame = cv2.resize(frame, (1000, 600))
            cv2.imshow("Object Detection in Video", frame)
        
            if cv2.waitKey(2) & 0xff==ord('q'):
                break
        
        capt.release()
        cv2.destroyAllWindows()


#Create label
video_label=widgets.HTML(
    value="<h2>Object Detection in VIDEO</h2>"
)

# Create video upload button
video_upload_button = widgets.FileUpload(accept='.mp4', description='Upload Video')


# Create detect video button
detect_video_button = widgets.Button(
    description=' Detect in Video',
    disabled=False,
    button_style='Danger',
    tooltip='Detect Video',
    icon='video' # (FontAwesome names without the `fa-` prefix)
)
detect_video_button.on_click(handle_detect_video_button_click)

# Create loader for video
video_loader = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Uploading Video...',
    bar_style='danger',
    style={'bar_color': '#ffff00', 'description_width': 'initial'},
    orientation='horizontal',
    format='<font color="black">%(value)d%%</font>'  # Show progress percentage in black font color
)


# Create container widget for video-related components
video_tab_content = widgets.VBox([video_label,video_upload_button, video_loader, widgets.HBox([detect_video_button], layout=widgets.Layout(margin='10px 0px 0px 0px'))])

# Attach event handlers to the video upload button
video_upload_button.observe(handle_video_upload, names='value')


# Webcam Detection--------------------------------
# Create label for webcam detection
webcam_label = widgets.HTML(value="<h2>Webcam Detection</h2>")

# Create button to open webcam
open_webcam_button = widgets.Button(
    description="Open Webcam",
      button_style='warning',
    tooltip='Open Webcam',
    icon='camera' # (FontAwesome names without the `fa-` prefix)
)

# Function to open webcam
def open_webcam(b):
    # Create VideoCapture object for webcam
    capt = cv2.VideoCapture(0)  # Open default webcam (change to 1 if needed)
    
    # Check if webcam opened correctly
    if not capt.isOpened():
        raise IOError("Cannot open Webcam")
    
    # Function to display webcam feed with object detection
    def display_webcam_with_detection():
        ret, frame = capt.read()
        if ret:
            ClassIndex, confidence, bbox = model.detect(frame, confThreshold=0.55)  # Perform object detection
            if len(ClassIndex) != 0:
                for ClassInd, conf, boxes in zip(ClassIndex.flatten(), confidence.flatten(), bbox):
                    if ClassInd <= 80:
                        cv2.rectangle(frame, boxes, (255, 0, 0), 2)  # Draw rectangle around detected object
                        cv2.putText(frame, classLabels[ClassInd-1], (boxes[0]+10, boxes[1]+40), cv2.FONT_HERSHEY_PLAIN, 3, (0, 255, 0), 3)  # Display class label
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            cv2.imshow("Object Detection in Webcam", frame)  # Display webcam feed with object detection
        
        # Check for key press
        key = cv2.waitKey(1)
        if key == ord('q'):  # Check if 'q' key is pressed
            cv2.destroyAllWindows()  # Close OpenCV windows
            capt.release()  # Release VideoCapture object
            return
    
    # Display webcam feed with object detection
    display_webcam_with_detection()
    
    # Update webcam feed continuously with object detection
    while capt.isOpened():
        display_webcam_with_detection()
    
# Attach event handlers to the buttons
open_webcam_button.on_click(open_webcam)

# Define a container for the webcam label, open webcam button, close webcam button, and webcam output
webcam_controls = widgets.VBox([webcam_label,open_webcam_button])


# LOGOUT SECTION--------------------------------
# Create label for Logout
logout_label = widgets.HTML(value="<h2><span style='font-size: 24px;'>&#x1F44B;</span> See You Again!!! <span style='font-size: 24px;'>&#x1F44B;</span></h2>")

# Create button to logout
logout_button = widgets.Button(
    description="Logout",
      button_style='danger',
    tooltip='Logout',
    icon='sign-out' # (FontAwesome names without the `fa-` prefix)
)

# Function to logout
def logout_function(b):
    # global uploaded_image_content
    # global uploaded_video_content

    # Clear uploaded image and video content
    # uploaded_image_content = None
    # uploaded_video_content = None
    
    # print("Logout! 😀")  # Emoji for welcome 
    # time.sleep(3)
    
    # Clear username and password text boxes
    username_textbox.value = ''
    password_textbox.value = ''

    # Reset file upload widgets
    image_upload_button.value = ()
    video_upload_button.value = ()

     # Clear output widget content
    with uploaded_output:
        clear_output()

    with captured_output:
        clear_output()

    # Reset loader widget
    loader.value = 0
    loader.description = 'Uploading Image...'

    video_loader.value = 0
    video_loader.description = 'Uploading Video...'
    
    # Clear login status output
    with login_status_output:
        clear_output()

    # Show the login tab and hide other tabs
    login_ui.layout.display = 'block'
    tabs_container.layout.display = 'none'

    # Show the image tab and hide other tabs
    tabs_container.selected_index = 0  # Index of the image tab in the tabs container
          
    
# Attach event handlers to the buttons
logout_button.on_click(logout_function)

# Define a container for the logout button
logout_content = widgets.VBox([logout_label,logout_button])


# Function to authenticate login++++++++++++++++++++++++++++++++
def authenticate_login(username, password):
    # Replace this with your actual login authentication logic
    if username == "admin" and password == "admin":
        return True
    else:
        return False

# Username and password text boxes
username_textbox = widgets.Text(placeholder="Enter username")
password_textbox = widgets.Password(placeholder="Enter password")

# Login button
login_button = widgets.Button(description="Login", button_style="success")

# Output widget for displaying login status
login_status_output = widgets.Output()

# Function to handle login button click
def handle_login_button_click(b):
    with login_status_output:
        clear_output()
        username = username_textbox.value.strip()
        password = password_textbox.value.strip()
        
        if authenticate_login(username, password):
            print("😀 Login Successful! 😀")
            time.sleep(2)
         
            # Show other tabs and hide the login tab 
            login_ui.layout.display = 'none'
            tabs_container.layout.display = 'block'
                
            # Show the image tab and hide other tabs
            tabs_container.selected_index = 0  # Index of the image tab in the tabs container
                        
        else:
            print("Invalid username or password. Please try again.")

# Attach event handler to the login button
login_button.on_click(handle_login_button_click)

# Arrange widgets in a VBox layout
login_ui = widgets.VBox([
    widgets.HTML(value="<h3>Welcome to Login Panel</h3>"),
    widgets.HBox([widgets.Label(value="Username:"), username_textbox]),
    widgets.HBox([widgets.Label(value="Password :"), password_textbox]),
    login_button,
    login_status_output
])

# Create tabs for image, video, and webcam-------------------+++++++++++++
tab_contents = ['Image', 'Video','Webcam','Logout']
tabs = widgets.Tab()
tabs.children = [image_tab_content, video_tab_content,webcam_controls,logout_content]
for i, tab_name in enumerate(tab_contents):
    tabs.set_title(i, tab_name)


# Set height of the entire tab widget
tabs_container = widgets.VBox([tabs], layout={'height': '1000px'})

# Display the login UI
display(login_ui)

# Display the tabs
display(tabs_container)

# Initially, only show the login tab
tabs_container.layout.display = 'none'

# Set the height of the output cell
display(HTML("<style>.output { height: 1200px; }</style>"))